# Requirement 1
- NAME = 'Abdelaziz Salah'
- BN = '3' 
- SEC = '2'
* In this requirement we are asked to apply laser-range mesaurments for an opening angle of 250, every 2 degrees, with maximum range of 12 meters.
* assuming that every pixel has a size of 4x4 cm2


In [1]:

import cv2
from typing import Tuple,List
from math import sin,cos,pi
from skimage.draw import line as line_points
import numpy as np
from numpy.linalg import norm



In [2]:
WHITE = (255,255,255)
BLUE = (255,0,0)
GREEN = (0,255,0)
BLACK = (0,0,0)
ROBOT_BORDER=(213,115,80)
FILL = -1
ROBOT_RADIUS = 10
ROBOT_BORDER_THICKNESS = 2
DIRECTION_LINE_LENGTH = 20
RADIAN = pi/180.0
SENSOR_ANGLE = 250
SENSOR_RESOLUTION = 2

In [3]:
def farestPointFromCertainXYTheta(r:float,theta: float, x: int, y:int) -> Tuple[int,int]:
    '''
        INPUT:
            x,y -> starting point of a line
            r -> the length of the line
            theta -> the angle of the line
            RETURNS:
                 farest point from the starting point in the direction of the angle
    '''
    x2 = int(x + r * cos(theta *RADIAN))
    y2 = int(y + r * sin(theta *RADIAN))

    return x2,y2

def drawLine(
         startingPoint: Tuple[int,int],
         lineAngle : float,
         lineLength: float,
         thickness: int,
         color: Tuple[int,int,int],
         img: cv2.Mat,
         ) -> cv2.Mat:
    '''
        INPUT:
            startingPoint -> starting point of a line
            lineLength -> the length of the line
            color -> the color of the line
            thickness -> the thickness of the line
            lineAngle -> the angle of the line
            img -> the image to draw on
            RETURNS:
                 the image with the line drawn
                     
    '''
    x,y = startingPoint
    x2,y2 = farestPointFromCertainXYTheta(lineLength,lineAngle,x,y)
    return cv2.line(img,startingPoint,(x2,y2),color,thickness)

def cast_ray(img: cv2.Mat,
             startingPoint: Tuple[int,int],
             maxLineLength: int,
             lineAngle: float,
             color: Tuple[int,int,int]) -> Tuple[int,int]:
    '''
        INPUT:
            startingPoint -> starting point of a line
            maxLineLength -> the max length of the line (12m)
            color -> the color of the line
            lineAngle -> the angle of the line
            img -> the image to draw on
            RETURNS:
                 the image with a line drawn on it
    '''
    x1,y1 = startingPoint

    x2,y2 = farestPointFromCertainXYTheta(maxLineLength,lineAngle,x1,y1)
     

    # get all the points on a certain line.
    pointsOnLine = list(zip(*line_points(x1,y1,x2,y2)))

    endpoint = None
    done = False
    for x,y in pointsOnLine:
        if norm(img[y,x] - np.array(BLACK)) == 0 :
            done = True
            endpoint = (x,y)
            break
        if not done: endpoint = (x,y)
       
    
    img = cv2.line(img,(x1,y1),endpoint,color,thickness=1)
    return endpoint

def draw_robot(map, x, y, t):
    '''
        INPUT:
            x,y -> starting point of a line
            t -> the angle of the line
            map -> the image to draw on
            RETURNS:
                 the image with the robot drawn
    '''
    img = cv2.circle(map,
                    center= (x,y),
                    radius= ROBOT_RADIUS,
                    color = BLUE,
                    thickness=FILL)
    img = cv2.circle(img,
                    center= (x,y),
                    radius= ROBOT_RADIUS,
                    color = ROBOT_BORDER,
                    thickness=ROBOT_BORDER_THICKNESS)
    img = drawLine(
        img=img,
            startingPoint= (x,y),
                lineLength= DIRECTION_LINE_LENGTH,
                lineAngle= t,
                color= ROBOT_BORDER,
                thickness= ROBOT_BORDER_THICKNESS)
    return img

def move_ray(distance: int, x: float, y: float, angle: int):
    '''
        INPUT:
            x,y -> starting point of a line
            distance -> the length of the line
            angle -> the angle of the line
            RETURNS:
                new line after moving distance in the angle direction
    '''
    return round(x + distance * np.cos(angle * np.pi / 180)), round(y + distance * np.sin(angle * np.pi / 180))

In [4]:
def requirement1(image_path: str , x: int, y: int, t: int):
    '''
        This function is for requirement 1

        Parameters:
            image_path: str
                Path to the image
            x,y: int
                Location of Robot, (0,0) is the left-top corner of the 2D map, so x increases to the right and y increases to the down.
            t: int
                Angle of Robot's pose, 0 means robot is facing to the right, 90 means to the down, 180 to the left, 270 to the up.
    '''
    # Reading an image in color mode
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)

    # write the position of the robot in txt file
    file = open("data.txt", "w")
    file.write("Robot location: " + "x: " + str(x) + " y: " + str(y) + " theta: " + str(t))

    # draw the robot
    image = draw_robot(image, x, y, t)

    # create array to save the distance at each angle
    distances = np.zeros(int(SENSOR_ANGLE/SENSOR_RESOLUTION)+1)
    endpoints = []

    index = 0

    # write meaeurements in txt file
    file.write("\n\nMeasurements: ") 
    file.write("\nAngle\t\tDistance\t\tEndpoint")


    for angle in range(t- int(SENSOR_ANGLE/ 2),t+ int(SENSOR_ANGLE/2) + 1,2):
        endpoint = cast_ray(img = image,
                            startingPoint= (x,y),
                            maxLineLength= int(12 *100 / 4), # 12 * 100 to convert it to cm, then divide by 4 as each pixel is 4x4 cm
                            lineAngle= angle,
                            color= GREEN)
                            

        # calculate the distance between the robot and the endpoint
        distance = norm(np.array((x,y)) - np.array(endpoint))

        # save the distance in the array
        distances[index] = min(1200, distance * 4) #? 4 for each pixel
        endpoints.append(endpoint)

        # write meaeurements in txt file
        file.write("\n" + str(angle) + "\t\t\t" + str(round(distances[index],3)) + "\t\t\t" + str(endpoint))
        
        # increase the index
        index += 1
    
    file.close()

    cv2.imwrite('laserRange.jpg', image)

    return distances , np.array(endpoints) 


requirement1('map.jpg', 80, 100, 0)

(array([ 103.2279032 ,  109.83624174,  116.6190379 ,  123.54756169,
         130.59862174,  141.36477638,  152.31546212,  163.41358573,
         182.16476059,  196.16319736,  405.56133938,  402.65121383,
         399.29938643,  397.19013079,  394.92784151,  393.64959037,
         392.50987249,  392.08162415,  392.08162415,  392.50987249,
         393.64959037,  394.92784151,  396.56525314,  399.29938643,
         401.75614494,  405.56133938,  409.95609521,  414.92167936,
         419.00835314,  426.48329393,  433.03579529,  434.69529558,
         411.82520564,  387.5874095 ,  367.23834222,  347.35572545,
         331.20386471,  318.59692403,  306.30703551,  294.37391189,
         282.84271247,  274.48861543,  263.7877935 ,  256.1249695 ,
         251.20509549,  244.13111231,  237.45315327,  233.23807579,
         229.22478051,  223.60679775,  220.14540649,  216.92394981,
         213.95326593,  211.24393482,  208.80613018,  206.64946165,
         205.67936211,  203.96078054,  202.54382